# TMA Analysis code supporting SITCOM-1310
Craig Lage - 07-Apr-24

During several nights, the TMA was set to track for long period, form 10 minutes to over an hour.  Then the StarTrackers were used to determine how well the TMA was tracking the sky.  This long term drift could be due to TMA deficiencies, or to deficiencies in the pointing model.

# Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
dataDir = Path("./plots")
dataDir.mkdir(exist_ok=True, parents=True)

# These are the nights when the tests were run
dayObss = [20240403, 20240404, 20240405]

# Some sequences give NaNs for some reason.  Skipping those.
# This is the dayObs, and the first seqNum of the exposure series.
# These seem to be ones at the beginning or end of the night
onesToSkip = [[20240403, 98], [20240404, 15], [20240404, 17], [20240404, 1042]]

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.summit.utils.utils import dayObsIntToString
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, getAzimuthElevationDataForEvent, filterBadValues

%matplotlib inline

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

# This uses the json metadata from RubinTV, which contains the astrometric solutions for the StarTracker exposures.  You need to download the metadata for each dayObs and put it in a location where the program can find it.

In [ ]:
prePad = 10.0
postPad = 10.0
pdf = PdfPages(dataDir / "Long_Term_Tracking_Events_06Apr24.pdf")
fig = plt.figure(figsize=(15,6))

# The lists below will be used to lot the 
# tracking drif vs AltAz
alts = []
azs = []
raDrifts = []
decDrifts = []

for dayObs in dayObss:
    dayObsString = dayObsIntToString(dayObs)
    filename = f'../json_metadata/startracker_narrow_{dayObsString}.json'
    jsonData = pd.read_json(filename).T
    jsonData = jsonData.sort_index()
    events = eventMaker.getEvents(dayObs)
    longTracks = [e for e in events if (e.type.name == 'TRACKING' and e.duration > 500)]
    
    for n, track in enumerate(longTracks):
        seqNums = []
        for i in range(len(jsonData)):
            try:
                time = Time(jsonData.iloc[i]['MJD narrow'], format='mjd').unix_tai - 37.0
                if time > track.begin.value + prePad and time < track.end.value - postPad:
                    seqNums.append(i)
            except:
                continue
        print(n, len(seqNums))
        if len(seqNums) == 0:
            continue
        
        azimuthData, elevationData = getAzimuthElevationDataForEvent(client, track)
        azTimes = azimuthData['timestamp'].values
        azTimes -= azTimes[0]
        azErrors = azimuthData['azError'].values
        filterBadValues(azErrors, maxConsecutiveValues=10)
        elTimes = elevationData['timestamp'].values
        elTimes -= elTimes[0]
        elErrors = elevationData['elError'].values
        filterBadValues(elErrors, maxConsecutiveValues=10)
    
        times = []
        ras = []
        decs = []
        for seqNum in seqNums:
            try:
                time = Time(jsonData.iloc[seqNum]['MJD narrow'], format='mjd').unix_tai
                ra = jsonData.iloc[seqNum]['Calculated Ra narrow']
                dec = jsonData.iloc[seqNum]['Calculated Dec narrow']
                ras.append(ra)
                decs.append(dec)
                times.append(time)
            except:
                continue

        if [dayObs, seqNums[0]] in onesToSkip:
            continue
        azs.append(np.median(azimuthData['actualPosition'].values))
        alts.append(np.median(elevationData['actualPosition'].values))
        
        times = np.array(times)
        times -= times[0]
        ras = np.array(ras)
        ras = (ras - ras[0]) * 3600.0
        decs = np.array(decs)
        decs = (decs - decs[0]) * 3600.0
        raDrift = (ras[-1] - ras[0]) / times[-1] * 30.0
        decDrift = (decs[-1] - decs[0]) / times[-1] * 30.0
        raDrifts.append(raDrift)
        decDrifts.append(decDrift)
        axs = fig.subplots(1,3)
        plt.suptitle(f"Long term tracking drift {dayObsString}: {seqNums[0]} - {seqNums[-1]}  RA = {ra:.2f}, Dec = {dec:.2f}", \
                     fontsize=18)
        plt.subplots_adjust(wspace=0.3)
        axs[0].set_title(f"RA drift = {raDrift:.2f} arcseconds/30 seconds")
        axs[0].scatter(times, ras)
        axs[0].set_ylabel("RA Drift (arcseconds)")
        axs[0].set_xlabel("Time (seconds)")
        axs[1].set_title(f"Dec drift = {decDrift:.2f} arcseconds/30 seconds")
        axs[1].scatter(times, decs)
        axs[1].set_ylabel("Dec Drift (arcseconds)")
        axs[1].set_xlabel("Time (seconds)")
        axs[2].set_title("Encoder tracking errors")
        axs[2].plot(azTimes, azErrors, color='green', label='Az')
        axs[2].plot(elTimes, elErrors, color='red', label='El')
        axs[2].set_ylabel("Errors (arcsec)")
        axs[2].set_xlabel("Time (sec)")
        axs[2].set_ylim(-1.0,1.0)
        axs[2].legend()
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
pdf.close()


# This cell plots the drift errors vs AltAz, using the size of the plot marker to indicate the tracking drift.

In [ ]:
radAzs = np.radians(azs)
fig=plt.figure(figsize=(16,8))
ax1 = plt.subplot(121, projection='polar')
ax1.invert_yaxis()
ax1.set_title("RA drift - Size proportional to errors\n Blue = pos, Red = Neg", fontsize=18)
radAzsPos = []
radAzsNeg = []
altsPos = []
altsNeg = []
raDriftsPos = []
raDriftsNeg = []
for n, raDrift in enumerate(raDrifts):
    if raDrift > 0.0:
        radAzsPos.append(radAzs[n])
        altsPos.append(alts[n])
        raDriftsPos.append(raDrift)
    else:
        radAzsNeg.append(radAzs[n])
        altsNeg.append(alts[n])
        raDriftsNeg.append(raDrift)
raSizePos = abs(np.array(raDriftsPos)) * 500.0
raSizeNeg = abs(np.array(raDriftsNeg)) * 500.0

ax1.scatter(radAzsPos, altsPos, color='blue', marker='o', s=raSizePos)
ax1.scatter(radAzsNeg, altsNeg, color='red', marker='o', s=raSizeNeg)

ax2 = plt.subplot(122, projection='polar')
ax2.invert_yaxis()
ax2.set_title("Dec drift - Size proportional to errors\n Blue = pos, Red = Neg", fontsize=18)
radAzsPos = []
radAzsNeg = []
altsPos = []
altsNeg = []
decDriftsPos = []
decDriftsNeg = []
for n, decDrift in enumerate(decDrifts):
    if decDrift > 0.0:
        radAzsPos.append(radAzs[n])
        altsPos.append(alts[n])
        decDriftsPos.append(decDrift)
    else:
        radAzsNeg.append(radAzs[n])
        altsNeg.append(alts[n])
        decDriftsNeg.append(decDrift)
decSizePos = abs(np.array(decDriftsPos)) * 500.0
decSizeNeg = abs(np.array(decDriftsNeg)) * 500.0
ax2.scatter(radAzsPos, altsPos, color='blue', marker='o', s=decSizePos)
ax2.scatter(radAzsNeg, altsNeg, color='red', marker='o', s=decSizeNeg)
plt.savefig(dataDir / "Long_Term_Tracking_Drifts_AltAz_06Apr24.pdf")